## **Training the model with MNIST dataset**

#### Import Libraries

In [10]:
import tensorflow as tf
import cv2 as cv
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
import numpy as np
from sklearn.metrics import accuracy_score

#### Load MNIST dataset

In [11]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

print(f'-------------------------\n')
print(f'Matrix X_train contains {X_train.shape[0]} images (resolution = {X_train.shape[1]} X {X_train.shape[2]})\n')
print(f'For all images X_train there are {y_train.shape[0]} labels, that goes from {min(y_train)} to {max(y_train)}\n')
print(f'-------------------------\n')
print(f'Matrix X_test contains {X_test.shape[0]} ({X_test.shape[1]} X {X_test.shape[2]} resolution)\n')
print(f'For all images X_test there are {y_test.shape[0]} labels, that goes from {min(y_test)} to {max(y_test)}\n')
print(f'-------------------------\n')
print(f'Each model has {X_train[0].shape[0]} x {X_train[0].shape[1]} = {X_train[0].shape[0]*X_train[0].shape[1]} features')
print(f'-------------------------\n')
print(f'Values from X_train and X_test goes from {np.min(X_train[0])} to {np.max(X_train[0])}\n')
print(f'-------------------------\n')

X_train = X_train / np.max(X_train[0])  # now values from 0 to 1
X_test = X_test / np.max(X_test[0])

-------------------------

Matrix X_train contains 60000 images (resolution = 28 X 28)

For all images X_train there are 60000 labels, that goes from 0 to 9

-------------------------

Matrix X_test contains 10000 (28 X 28 resolution)

For all images X_test there are 10000 labels, that goes from 0 to 9

-------------------------

Each model has 28 x 28 = 784 features
-------------------------

Values from X_train and X_test goes from 0 to 255

-------------------------



#### Create Models

In [12]:
model_1 = Sequential(
    [
    Flatten(input_shape=(28,28)),
    Dense(128,activation='relu'),
    Dense(10,activation='softmax'),
    ],
    name = 'model_1'
)

model_2 = Sequential(
    [
        Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1)),  # 28,28,1 (gray scale images 28x28)
        Conv2D(32, kernel_size=(3,3)),
        MaxPooling2D(pool_size=(2,2)),
        Conv2D(64, kernel_size=(3,3)),
        Conv2D(64, kernel_size=(3,3)),
        MaxPooling2D(pool_size=(2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(10,activation='softmax'),
    ],
    name = 'model_2'
)

In [13]:
model = model_2

In [14]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                       

In [15]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    # loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    optimizer = 'Adam',
    # optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy'],
)

history = model.fit(
    X_train,
    y_train,
    epochs = 15,
    validation_split = 0.2,
)

Epoch 1/15
1500/1500 [==============================] - 43s 28ms/step - loss: 0.1636 - accuracy: 0.9485 - val_loss: 0.0603 - val_accuracy: 0.9828
Epoch 2/15
1500/1500 [==============================] - 43s 29ms/step - loss: 0.0726 - accuracy: 0.9774 - val_loss: 0.0551 - val_accuracy: 0.9847
Epoch 3/15
1500/1500 [==============================] - 41s 28ms/step - loss: 0.0583 - accuracy: 0.9815 - val_loss: 0.0513 - val_accuracy: 0.9855
Epoch 4/15
1500/1500 [==============================] - 43s 29ms/step - loss: 0.0489 - accuracy: 0.9848 - val_loss: 0.0395 - val_accuracy: 0.9881
Epoch 5/15
1500/1500 [==============================] - 45s 30ms/step - loss: 0.0452 - accuracy: 0.9857 - val_loss: 0.0416 - val_accuracy: 0.9875
Epoch 6/15
1500/1500 [==============================] - 40s 27ms/step - loss: 0.0408 - accuracy: 0.9871 - val_loss: 0.0475 - val_accuracy: 0.9865
Epoch 7/15
1500/1500 [==============================] - 42s 28ms/step - loss: 0.0389 - accuracy: 0.9883 - val_loss: 0.0527 -

In [16]:
y_prob = model.predict(X_test)
y_hat = y_prob.argmax(axis=1)

313/313 [==============================] - 2s 6ms/step


In [17]:
y_hat = y_hat.astype(np.uint8)
accuracy_score(y_test,y_hat)

0.9834

In [19]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.0831 - accuracy: 0.9834


[0.08305638283491135, 0.9833999872207642]

In [20]:
model.save('CNN.hdf5')